## 0 | Import packages

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import tkinter
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from utilities import importFile, openFile, openHDF5file, getLooseRseal
from tkinter.filedialog import askopenfilename, askopenfilenames, askdirectory
from collections import defaultdict
from nptdms import TdmsFile
from scipy import stats
from scipy.signal import find_peaks
print("done!")

## 2 | Loop over files in a folder and find spikes

In [ ]:
test = askopenfilenames()
test

In [ ]:
os.listdir(folder_name_1)

In [ ]:
from tkinter.filedialog import askdirectory
folder_path = askdirectory()
list_of_files = os.listdir(folder_path)

# To loop over files we would
for file in list_of_files:
    import file
    find spikes 
    find Rseal
    save data

In [ ]:
# Load data
channels_data_frame, time, dt, folder_name, file_name = importFile(curated_channel = 'Sweeps_Analysis')
print("file imported")

In [ ]:
channels_data_frame

In [ ]:
# pandas.DataFrame.columns - The column labels of the DataFrame.
print(channels_data_frame.columns)

# pandas.DataFrame.index - The index (row labels) of the DataFrame.
print(channels_data_frame.index)

# pandas.DataFrame.iloc - integer-location based indexing for selection by position.
channels_data_frame.loc[['Channel A', 'Channel B'],:]

In [ ]:
for sweep in channels_data_frame.columns:
    sweep_IB = np.array(channels_data_frame.at['Channel B', sweep])

    peaks, properties = find_peaks(-sweep_IB,
                                   height = None,
                                   threshold = None,
                                   distance = 5,
                                   prominence = 50,
                                   width = 2,
                                   wlen = None,
                                   rel_height = None,
                                   plateau_size = None)
                                

In [ ]:
    cell_name = [file_name.split('.')[0]] # Get the file name without the extension
    print(cell_name)
    seal_resistance = []
    trial_keys = []

    for sweep in channels_data_frame.columns:
        
        sweep_IA = np.array(channels_data_frame.at['Channel A', sweep])
        sweep_IB = np.array(channels_data_frame.at['Channel B', sweep])
        sweep_OA = np.array(channels_data_frame.at['Output A', sweep])

        # Get the indices of the test_pulse using the Output Channel
        test_pulse = np.where(sweep_OA < 0)
        test_pulse_OA_indices = test_pulse[0]

        # # Find the edges of the test_pulse
        # test_pulse_start = np.where(np.diff(sweep_OA) < (0))
        # test_pulse_end = np.where(np.diff(sweep_OA) > (0))
        # # Extract them from the tuple
        # test_pulse_start_i = test_pulse_start[0][0]
        # test_pulse_end_i = test_pulse_end[0][0]

        # Use the indices of the test_pulse command to define baseline period and test period
        sweep_OA_baseline = np.mean(sweep_OA[:(test_pulse_OA_indices[0]-1)]) # -1 to stop baseline before command starts
        sweep_OA_pulse = np.mean(sweep_OA[test_pulse_OA_indices])
        test_pulse_command = sweep_OA_baseline - sweep_OA_pulse # mV

        # # Use the indices of the test_pulse command to define baseline period and test period
        # sweep_OA_baseline = np.mean(sweep_OA[int(test_pulse_start_i - (20/0.04)):test_pulse_start_i])
        # sweep_OA_pulse = np.mean(sweep_OA[int(test_pulse_end_i - (49/0.04)):test_pulse_end_i])
        # test_pulse_command = sweep_OA_baseline - sweep_OA_pulse #mV


        # Use the test_pulse indices to get the baseline and cell response to calculate the seal resistance
        # To be exact and account for the delays between digital command and output from the amplifier, you could add +1 to the first index to calculate the baseline.
        sweep_IB_baseline = np.mean(sweep_IB[:(test_pulse_OA_indices[0])])
        # Similary, to avoid using the values recorded while the test pulse command begins, you can skip a milisecond (+4 indices) to the beginning, to ensure you start averaging once the signal has reached the cell. To be extra exact, you could add +2 to the last index so you use all the samples. However, this shouldn't make a difference, so we just skip the milisecond to avoid the transition period.
        sweep_IB_pulse = np.mean(sweep_IB[(test_pulse_OA_indices[0]+4):(test_pulse_OA_indices[-1])])
        test_pulse_membrane = sweep_IB_baseline - sweep_IB_pulse # pA

        # # Do the same as above but for the Current channel
        # sweep_IB_baseline = np.mean(sweep_IB[int(test_pulse_start_i - (20/0.04)):test_pulse_start_i])
        # sweep_IB_pulse = np.mean(sweep_IB[int(test_pulse_end_i - (49/0.04)):test_pulse_end_i])
        # test_pulse_membrane = sweep_IB_baseline - sweep_IB_pulse #pA

        # Get seal resistance = mV/pA
        Rseal = (test_pulse_command / test_pulse_membrane) * 1000 # to get MOhm
        
        # append results
        seal_resistance.append(Rseal)
        trial_keys.append(sweep)

    # Create data frame of data:
    extracted_Rseal_data_frame = pd.DataFrame([seal_resistance], index = cell_name, columns = trial_keys)


## 3 | Create a data frame to save the results


Think about all the data we will need to export and save and create the backbone of the data frame that will contain it. Things to save:

 * 'sweep_data'
 * 'test_pulse_command'
 * 'test_pulse_membrane'
 * 'seal_resistance'
 * 'peaks'
 * 'n_spikes'
 * 'time_recording'
 * 'interspike_interval'
 * 'holding_isi'
 * 'holding_spikes'
 * 'cut_spikes'


Will also need to save prominences and the histograms used to select it, and we probably need to store std for each average.


### Types of firing rate to assess

We could take a look at the following:

 * Instantaneous firing rate: inverse of the interspike interval
 * Firing rate over full recording
 * Firing rate over time windows (1s?)

In [ ]:
cell_id = [file_name_1.split('.')[0]] # Get the file name without the extension

save_folder_name = str(folder_name_1 + '/results') # Set the path to save the file
results_dataframepd = .DataFrame(np.zeros(11),l coumns = ['sweep_number'], index =[''sweep_data'', test_pulse_command', 'test_pulse_membrane', 'sel_resistance, ]'peaks', 'n_spikes', 'time_recording', 'interspike_interval', 'holding_isi', 'holding_spikes', 'cut_spik]'
fil

results_datafme
# r Use pd.DataFrame.to_csv()  as file, with filename = dlpag_vgat_201201_c5...to save